In [2]:
from KGLE_Dataset import *
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
import random

In [3]:
torch.manual_seed(42)
np.random.seed(0)
# torch.use_deterministic_algorithms(True)
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [4]:
train_set = KGLE_Dataset("./data/processed_trainv2.csv")
train_load = DataLoader(dataset=train_set,
                        batch_size=32,
                        num_workers=8,
                        shuffle=True,
                        worker_init_fn=seed_worker,
                        generator=g,)

# WARNING: validation set is in an older, incompatible format to training set
valid_set = KGLE_Dataset("./data/processed_validv2.csv")
valid_load = DataLoader(dataset=valid_set,
                        batch_size=32,
                        num_workers=8,
                        shuffle=True,
                        worker_init_fn=seed_worker,
                        generator=g,)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [52]:
class TravelRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed_origin_stand: nn.Module = nn.Embedding(64, 3)
        self.embed_taxi_id: nn.Module = nn.Embedding(448, 5)
        self.feed_forward = nn.Sequential(
            nn.Linear(18, 1000),
            #             nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000, 800),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(800, 1)
        )

    def forward(self, input: torch.Tensor):
        origin_stand = self.embed_origin_stand(input[:, 1].to(dtype=torch.int32))
        taxi_id = self.embed_taxi_id(input[:, 2].to(dtype=torch.int32))
        input = torch.cat((input[:,:1], origin_stand, taxi_id, input[:, 3:]), dim=1).to(dtype=torch.float32)
        input = self.feed_forward(input)
        return input

model = TravelRegressor().to(device)

In [53]:
model.eval()
model(train_set[0][0].unsqueeze(0).to(device)).size()
# train_set[0][0].unsqueeze(0).size()

torch.Size([1, 1])

In [5]:
lossfn = nn.MSELoss(reduction="mean")
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
def weight_mse(pred, act):
    denom = torch.sum(act)
    return torch.sum(act * ((pred - act)**2 / len(act))) / denom

In [6]:
def validate():
    model.eval()
    loss = torch.tensor([0]).to(device, dtype=torch.float32)
    with torch.no_grad():
        for i, (entry, target) in enumerate(valid_load):
            entry = entry.to(device, dtype=torch.float32)
            target = target.to(device, dtype=torch.float32)
            preds = model(entry)
            loss += lossfn(preds, target)

    return loss /(i + 1)

In [7]:
def train(num_iter: int):
    for epoch in range(num_iter):
        for i, (entry, target) in enumerate(train_load):
            model.train()
            entry = entry.to(device, dtype=torch.float32)
            target = target.to(device, dtype=torch.float32)
            preds = model(entry)

            optimizer.zero_grad()
            loss = lossfn(preds, target) # or weighted mse
            loss.backward()
            optimizer.step()
            if (i + 1) % 10000 == 0 or i == 0:
                print(f"[Epoch: {epoch + 1}]\t[Iter: {i + 1}]\t[Weighted RMSE: {torch.sqrt(loss)}]")
                print(f"Prediction std dev: {torch.std(preds).item()}")

        valid_loss = validate()
        print(f"[RMSE Validation: {torch.sqrt(valid_loss).item()}]")